In [1]:
import pandas as pd

In [2]:
file = "data/4import_Database_Austria_March2018.xlsx"

In [3]:
def clean_coords(cord):
    lat = str(cord).replace(',', '.')
    lat = lat.replace('°', '')
    return float(lat)

In [4]:
df = pd.read_excel(file)[1:]

In [5]:
df

,*Site ID,Alternative Site ID,*Site Name,Alternative Site Name,*Country,Municipiality/community,Town,County,Cadastral Community,Cadastral Number,...,Description,Comment,Catchment Landscape,Literature,Public,EnteredBy,CheckedBy,Priority of entrance in database.,Coordinates N (Lat),Coordinates E (Long)
1,OE_08001,NaN,Königsberg,NaN,Austria,Styria,NaN,Leibnitz,Nestelberg bei Heimschuh,66147,...,Fortified hilltop settlement on top of the Kön...,NaN,NaN,NaN,yes,Raggam Patricia,NaN,1 - high,"46,763542°","15,461068°"
2,OE_08002,BorderArch: Leibnitz.66003.7; Leibnitz.66023.2...,Sulmtal Region,NaN,Austria,Styria,NaN,Leibnitz,Burgstall; \nMantrach; Großklein; Goldes; Glei...,66003; \n66023; \n66011;\n66010;\n66009,...,NaN,NaN,NaN,NaN,yes,Raggam Patricia,NaN,1 - high,"46,74701°","15,42075°"
3,OE_08003,Kiszter Tumuli Gruppe 35,Graschach Gleinstätten,NaN,Austria,Styria,NaN,Deutschlandsberg,Graschach,61153,...,NaN,NaN,NaN,NaN,yes,Raggam Patricia,NaN,1 - high,"46,756363°","15,350759°"
4,OE_08004,BorderArch: Deutschlandsberg.61119.1,Hörmsdorf,NaN,Austria,Styria,NaN,Deutschlandsberg,Hörmsdorf,61119,...,NaN,NaN,NaN,NaN,yes,Raggam Patricia,NaN,1 - high,"46,689317°","15,261181°"
5,OE_08005,BorderArch: Deutschlandsberg.61114.1,Feisternitz,NaN,Austria,Styria,NaN,Deutschlandsberg,Feisternitz,61114,...,NaN,NaN,NaN,NaN,yes,Raggam Patricia,NaN,1 - high,"46,68729°","15,26132°"
6,OE_08006,BorderArch: Deutschlandsberg.61112.1,Eibiswald,NaN,Austria,Styria,NaN,Deutschlandsberg,Eibiswald,61112,...,NaN,NaN,NaN,NaN,yes,Raggam Patricia,NaN,1 - high,"46,6818°",15.2314
7,OE_08007,NaN,Weniggleinz,NaN,Austria,Styria,NaN,Deutschlandsberg,Wettmannstätten,61069,...,NaN,NaN,NaN,NaN,yes,Raggam Patricia,NaN,2 - middle,46.8149,15.3812
8,OE_08008,BorderArch: Leibnitz.66019.1,Langegg,NaN,Austria,Styria,NaN,Leibnitz,Langegg,66019,...,NaN,NaN,NaN,NaN,yes,Raggam Patricia,NaN,3 - low,46.6504,15.5479
9,OE_08009,NaN,Schönegg,NaN,Austria,Styria,NaN,Leibnitz,Schönegg,66171,...,NaN,NaN,NaN,NaN,yes,Raggam Patricia,NaN,2 - middle,46.7422,15.5145
10,OE_08010,NaN,Hörbing,NaN,Austria,Styria,NaN,Deutschlandsberg,Hörbing,61025,...,NaN,NaN,NaN,NaN,yes,Raggam Patricia,NaN,2 - middle,46.8096,15.2351


In [6]:
other_period_scheme, _ = SkosConceptScheme.objects.get_or_create(dc_title="otherperiods")

In [7]:
failed = []
for i, row in df.iterrows():
    temp_site,_ = Site.objects.get_or_create(
        identifier=row['*Site ID']
    )
    temp_site.name=row['*Site Name']
    temp_site.cadastral_number=row['Cadastral Number']
    temp_site.plot_number=row['Plot Number']
    temp_site.description = row['Description\n']
    
    # related other_period
    
    if isinstance(row['Other present periods'], str):
        for y in row['Other present periods'].split(';'):
            temp_period, _ = SkosConcept.objects.get_or_create(
                pref_label=y,
                scheme__in=[other_period_scheme]
            )
            temp_site.other_period.add(temp_period)
    
    # related places:
    if isinstance(row['Cadastral Community'], str):
        temp_nation, _ = Place.objects.get_or_create(
                name=row['*Country']
            )
        temp_land, _ = Place.objects.get_or_create(
                name=row['Municipiality/community'],
                part_of=temp_nation
            )
        temp_county, _ = Place.objects.get_or_create(
                name=row['County'], part_of=temp_land
            )
        lat = clean_coords(row['Coordinates N (Lat)'])
        lng = clean_coords(row['Coordinates E (Long)'])
    
        for x in row['Cadastral Community'].split(";"):
            temp_place, _ = Place.objects.get_or_create(
                name=x, part_of=temp_county, lat=lat, lng=lng
            )
            temp_site.cadastral_community.add(temp_place)
            
            try:
                temp_site.save()
            except:
                pass
    if row['Public'] == "yes":
        temp_site.public = True
    try:
        temp_site.save()
    except:
        failed.append(temp_site)

In [ ]:
#for i, row in df.iterrows():
    enttype = row['ArchEntType']
    site, _ = Site.objects.get_or_create(
        identifier=row['*SiteID']
    )
    if isinstance(enttype, str):
        vocab, _ = SkosConcept.objects.get_or_create(
            pref_label = enttype
        )
        print(enttype)
        
        temp_ent, _ = ArchEnt.objects.get_or_create(
            identifier=row['*Entity ID'],
            site_id=site
        )
        temp_ent.name=row['Entity Name']
        temp_ent.comment=row['Comment']
        if row['Public'] == "yes":
            temp_ent.public = True
        temp_ent.ent_type = vocab
        temp_ent.save()
        
    else:
        print(site)